### 匹配特征值

In [7]:
import pandas as pd

def preprocess_valid_data(train_csv_path, valid_csv_path, output_valid_csv_path):
    try:
        # 加载训练数据
        train_data = pd.read_csv(train_csv_path, encoding='utf-8')
        print("训练数据导入成功")

        # 删除训练数据中的 'pointID' 和 'Y' 列，得到训练时使用的特征列
        X_train = train_data.drop(['pointID', 'Y'], axis=1)
        train_features = set(X_train.columns)
        print("训练数据特征列已提取")

        # 加载验证数据
        valid_data = pd.read_csv(valid_csv_path, encoding='utf-8')
        print("验证数据导入成功")

        # 删除验证数据中的 'pointID.1' 列，保留 'pointID' 列
        try:
            valid_data = valid_data.drop(['pointID.1'], axis=1)
            print("验证数据中的 'pointID.1' 列已删除")
        except KeyError as e:
            print(f"删除列时出错: {e}")

        # 找出验证数据中多出的列
        extra_columns = set(valid_data.columns) - train_features - {'pointID'}
        print(f"验证数据中多出的列: {extra_columns}")

        # 删除验证数据中多出的列
        if extra_columns:
            valid_data = valid_data.drop(columns=list(extra_columns))
            print(f"已删除验证数据中的多余列: {extra_columns}")

        # 保存处理后的验证数据，保留 'pointID' 列
        valid_data.to_csv(output_valid_csv_path, index=False, encoding='utf-8')
        print(f"处理后的验证数据已保存至 {output_valid_csv_path}")

    except Exception as e:
        print(f"处理验证数据时发生错误: {e}")

# 示例使用
train_csv_path = '训练数据删除2和4特征工程后\工业用地_修正_删除2和4_处理后.csv'  # 替换为您的训练数据路径
valid_csv_path = '再再进进一步核算\工业_修正.csv'  # 替换为您的验证数据路径
output_valid_csv_path = '对齐特征处理后的验证数据\工业用地.csv'  # 替换为您希望保存的输出路径

preprocess_valid_data(train_csv_path, valid_csv_path, output_valid_csv_path)


训练数据导入成功
训练数据特征列已提取
验证数据导入成功
验证数据中的 'pointID.1' 列已删除
验证数据中多出的列: {'距海关的距离_190207', '距游泳馆的距离_180111', '距客运汽车站的距离_230104', '距农家乐/民俗游的距离_180301', '距家用电器及电子产品零售的距离_130600', '距副食专卖店的距离_130207', '距自治区级道路的距离', '距科研机构的距离_240100', '距高速出入口的距离', '距村委会/居委会的距离_190106', '距游乐园的距离_180307', '距观光游览车站点的距离_180401', '距住宅楼的距离_120202', '距医疗机构的距离_1701', '距医疗器械的距离_130502', '距工艺礼品的距离_130407', '距烟酒副食、地方特产的距离_130200', '距眼镜店的距离_130302', '距风景名胜售票点的距离_210304', '距现状110k伏埋地电缆的距离', '距资料馆、档案馆的距离_160204', '汽车销售及服务POI核密度', '距公共设施的距离_19', '距其他单位的距离_220400', '距RD-01-高速路的距离', '距农贸市场的距离_130202', '距票、费服务的距离_2103', '距体育用品店的距离_130401', 'RD-03-1-干线主干路核密度', '距枢纽的距离_230228', '距其他单位的距离_2204', '距卫生、社保的距离_17', '轨道S4号线核密度', '距通信器材的距离_130603', '距地级市政府的距离_190103', '距居家服务的距离_2101', '距练歌房、KTV的距离_180201', '距南宁市现状公园绿地（已建成）的距离', '距中学的距离_160103', '距RD-04-公路的距离', '距轨道2号线的距离', '距国际组织的距离_1905', '距母婴用品、儿童用品零售的距离_1303', '距大厦的距离_200103', '距自行车、电动车、摩托车销售的距离_130804', '距党校的距离_160111', '距公路未分级的距离', '距轨道1号线的距离', '距厂家推荐二手车交易的距离_140303', '高压燃气管道路核密度', '现状电网

### 模型预测

In [8]:
import os
import pandas as pd
import joblib
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np

def evaluate_model(model_path, test_csv, output_csv):
    # 加载模型
    model = joblib.load(model_path)
    
    # 加载测试数据
    test_data = pd.read_csv(test_csv, encoding="utf-8")
    
    # 检查是否存在 pointID 列
    if 'pointID' in test_data.columns:
        test_data.set_index('pointID', inplace=True)
    else:
        # 自动查找类似的列名
        possible_cols = [col for col in test_data.columns if 'pointid' in col.lower()]
        if len(possible_cols) == 1:
            test_data.set_index(possible_cols[0], inplace=True)
        else:
            raise KeyError("无法找到 'pointID' 列，请检查 CSV 文件的列名。")

    # 删除 pointID.1 列
    if 'pointID.1' in test_data.columns:
        test_data = test_data.drop(['pointID.1'], axis=1)
    
    # 获取原始 pointID 列
    ID = test_data.index

    # 只保留模型训练时的特征列，删除多余的列
    # X_test = test_data[model.feature_names_in_]
    X_test = test_data
    # # 检查缺失的特征列，并填充缺失列
    # missing_cols = set(model.feature_names_in_) - set(X_test.columns)
    # for col in missing_cols:
    #     X_test[col] = 0  # 使用0填充缺失的特征列

    # 确保列顺序与训练数据一致
    # X_test = X_test[model.feature_names_in_]

    # 对测试数据进行标准化
    scaler = StandardScaler()
    X_test_standardized = scaler.fit_transform(X_test)
    X_test_standardized = pd.DataFrame(X_test_standardized, columns=X_test.columns)

    # 数据归一化
    normalizer = MinMaxScaler()
    X_test_normalized = normalizer.fit_transform(X_test_standardized)
    X_test_normalized = pd.DataFrame(X_test_normalized, columns=X_test.columns)

    # 使用模型进行预测
    y_test_pred = model.predict(X_test_normalized)
    
    # 保存结果，将索引恢复为列
    result = pd.DataFrame({"pointID": ID, "预测值": y_test_pred})
    result.to_csv(output_csv, index=False, float_format='%.6f', encoding='gbk')
    print(f"结果生成至 {output_csv}")

# 示例使用
model_path = "model\建设用地\单模型\工业用地\pkl\ExtraTreesRegressor.pkl"  # 模型文件路径
test_csv = '对齐特征处理后的验证数据\工业用地.csv'  # 测试数据文件路径
output_csv = './预测结果/工业用地.csv'  # 输出结果文件路径

evaluate_model(model_path, test_csv, output_csv)


结果生成至 ./预测结果/坑塘水面.csv


### 合并预测结果

In [ ]:
import os
import pandas as pd

def merge_csv_files(input_folder, output_file):
    # 获取输入文件夹中所有CSV文件的路径
    csv_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.csv')]
    
    # 初始化一个空的DataFrame用于合并
    merged_df = pd.DataFrame()

    # 遍历每个CSV文件
    for file in csv_files:
        # 读取CSV文件
        df = pd.read_csv(file, encoding='gbk')
        
        # 检查是否包含 'pointID' 列
        if 'pointID' not in df.columns:
            print(f"文件 {file} 不包含 'pointID' 列，跳过此文件。")
            continue
        
        # 将读取的DataFrame合并到总的DataFrame中
        merged_df = pd.concat([merged_df, df], ignore_index=True)
    
    # 按 'pointID' 列进行升序排列
    merged_df = merged_df.sort_values(by='pointID')

    # 保存合并后的数据
    merged_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"合并后的CSV文件已保存至 {output_file}")

# 示例使用
input_folder = '预测结果'  # 替换为包含多个CSV文件的文件夹路径
output_file = '核算.csv'  # 替换为您希望保存的输出文件路径

merge_csv_files(input_folder, output_file)


### 添加入shp文件的字段

In [ ]:
import pandas as pd
import geopandas as gpd

def add_csv_columns_to_shp(csv_file, shp_file, output_shp):
    # 读取CSV文件
    csv_data = pd.read_csv(csv_file, encoding='utf-8')
    
    # 读取SHP文件
    shp_data = gpd.read_file(shp_file)
    
    # 检查行数是否一致
    if len(csv_data) != len(shp_data):
        raise ValueError("CSV文件和SHP文件的行数不一致，无法合并。")
    
    # 将CSV文件的列逐列添加到SHP文件中
    for column in csv_data.columns:
        if column in shp_data.columns:
            print(f"列 {column} 在SHP文件中已存在，将覆盖该列数据。")
        shp_data[column] = csv_data[column].values  # 按顺序将CSV的列数据添加到SHP
    
    # 保存合并后的Shapefile
    shp_data.to_file(output_shp, driver='ESRI Shapefile', encoding='utf-8')
    print(f"CSV数据已成功添加到SHP文件中，并保存为 {output_shp}")

# 示例使用
csv_file = '合并后的CSV文件.csv'  # 替换为CSV文件路径
shp_file = '原始SHP文件.shp'  # 替换为SHP文件路径
output_shp = '合并后的SHP文件.shp'  # 替换为您希望保存的输出SHP文件路径

add_csv_columns_to_shp(csv_file, shp_file, output_shp)
